In [1]:
from datetime import datetime, timedelta
from pathlib import Path

from adcircpy import AdcircMesh, AdcircRun, Tides
from adcircpy.forcing.winds import BestTrackForcing
from adcircpy.utilities import download_mesh

import numpy as np

stage = 'staging'
HURRICANE = 'Matthew2016'
OUTPUT_DIRECTORY = stage+'/runs/'+HURRICANE+'_2'

In [2]:
# open mesh file
mesh = AdcircMesh.open('datasets/nc_inundation_v6c.grd', crs=4326)    

# Adding Manning's
# generate tau0 factor
mesh.generate_tau0()

# also add Manning's N to the domain (constant for this example)
mesh.mannings_n_at_sea_floor = np.full(mesh.values.shape, 0.025)

# initialize tidal forcing and constituents
tidal_forcing = Tides()
tidal_forcing.use_all()
mesh.add_forcing(tidal_forcing)

/global/home/users/alexandregeorges/.local/lib/python3.11/site-packages/adcircpy/mesh/parsers/grd.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['nodes'] = pandas.read_csv(
/global/home/users/alexandregeorges/.local/lib/python3.11/site-packages/adcircpy/mesh/parsers/grd.py:37: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['elements'] = pandas.read_csv(


In [3]:
# set simulation dates
spinup_time = timedelta(days=15)                                                                        
duration = timedelta(days=11)
start_date = datetime(2016, 9, 28, 12)
end_date = start_date + duration

In [4]:
# initialize wind forcing
wind_forcing = BestTrackForcing(storm=HURRICANE, nws=20)
mesh.add_forcing(wind_forcing)

/global/home/users/alexandregeorges/.local/lib/python3.11/site-packages/stormevents/nhc/storms.py:64: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  storms = pandas.read_csv(
/global/home/users/alexandregeorges/.local/lib/python3.11/site-packages/stormevents/nhc/track.py:1110: RuntimeWarning: invalid value encountered in divide
  speeds = pandas.Series(distances / abs(intervals), index=indices)


In [5]:
# instantiate driver object
driver = AdcircRun(mesh, start_date, end_date, spinup_time, netcdf=True)   
driver.NOUTGE = -3
driver.NOUTGV = -3
driver.NOUTGM = -3
driver.TOUTFGE = 20
driver.TOUTFGV = 20
driver.TOUTFGM = 20
driver.NSPOOLGE = 20
driver.NSPOOLGV = 20
driver.NSPOOLGM = 20
                                                                                                                                                  # write driver state to disk
driver.write(OUTPUT_DIRECTORY, overwrite=True)

/global/home/users/alexandregeorges/.local/lib/python3.11/site-packages/stormevents/nhc/track.py:641: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  atcf["background_pressure"].fillna(method="ffill", inplace=True)
/global/home/users/alexandregeorges/.local/lib/python3.11/site-packages/stormevents/nhc/track.py:641: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  atcf["background_pressure"].fillna(method="ffill", inplace=True)
syntax error, une

OSError: [Errno -70] NetCDF: DAP server error: b'https://icdc.cen.uni-hamburg.de/thredds/dodsC/ftpthredds/hamtide//k2.hamtide11a.nc'

In [ ]:
bash_script = rf"""
#!/bin/bash

# Rename and modify fort.15 for netcdf output and wind run
# mv fort.15.coldstart fort.15

# replacement_ele="-3 0.000000 15.000000 20                              ! NOUTGE"
# replacement_vel="-3 0.000000 15.000000 20                              ! NOUTGV"
rep_NWS="8                                                               ! NWS"
# storm_ref="2016 09 28 18 1 0.7 1                                           ! IREFYR IREFMO IREFDAY IREFHR StormNumber BLAdj GEOFACTOR"
# metstation="0 0 0 0                                                         ! NOUTM TOUTSM TOUTFM NSPOOLM         - MET STATION OUTPUT INFO (UNIT 71 and 72)"
# nstam="0                                                               ! NSTAM"
# wind_output="-3 0.000000 15.000000 20                              ! NOUTGW TOUTSGW TOUTFGW NSPOOLGW     - GLOBAL WIND STRESS AND ATM PRESSURE OUTPUT INFO (UNIT 73 and 74)"

# sed -i "s/0 0.000000 0.000000 0                                           ! NOUTGE/$replacement_ele/" fort.15
# sed -i "s/0 0.000000 0.000000 0                                           ! NOUTGV/$replacement_vel/" fort.15
sed -i "s/0                                                               ! NWS/$rep_NWS/" fort.15.hotstart
# sed -i "/0                                                               ! REFTIM/a\
# $storm_ref" fort.15
# sed -i "/110                                                             ! ANGINN/a\
# $metstation\n\
# $nstam" fort.15
# sed -i "/-3 0.000000 15.000000 20                              ! NOUTGV/a\
# $wind_output" fort.15


current_dir=$(pwd)
folder_name=$(basename $current_dir)

mkdir ../../logs
mkdir ../../logs/$folder_name
log_header="Logs of run attempts and their parameters."
log_header2="Hurricane Matthew2016"
log_header3="------------------------------------------"
log_header4="id\t\t\t\t\t\tdt\t\t\t\t\t\ttype\t\t\t\t\t\tcomment"
echo -e $log_header > runlog.out
echo -e $log_header2 >> runlog.out
echo -e $log_header3 >> runlog.out
echo -e $log_header4 >> runlog.out

cp ../../../scripts/fort22fix.py fort22fix.py
python fort22fix.py

# call aswip to format fort.22 proper
# aswip -n 20 -m 4 -z 2
# mv fort.22 ../../logs/$folder_name/fort.22
# mv NWS_20_fort.22 fort.22

cp ../../../scripts/run_new.sh run.sh
cp ../../../scripts/slurmcold.sh slurmcold.sh
cp ../../../scripts/slurmhot.sh slurmhot.sh

current_dir=$(pwd)
folder_name=$(basename $current_dir)
sed -i "s/^#SBATCH --job-name=.*$/#SBATCH --job-name=$folder_name/" slurmcold.sh
sed -i "s/^#SBATCH --job-name=.*$/#SBATCH --job-name=$folder_name hot/" slurmhot.sh

chmod +x run.sh slurmcold.sh slurmhot.sh
"""

# Write the bash script to the output directory
with open(OUTPUT_DIRECTORY + '/init.sh', 'w') as file:
    file.write(bash_script)

print("Bash script created and outputted at", OUTPUT_DIRECTORY)

Bash script created and outputted at staging/runs/Matthew2016_test
